In [1]:
import pandas as pd

'''DATA CLEAN-UP PART'''

titanic =pd.read_csv('/home/jimbrootan/Desktop/ML_data/Titanic_Data/train.csv')
titanic.shape
titanic=titanic.drop(['Ticket','Cabin'],axis=1)
titanic=titanic.dropna()
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,29.1250,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C


In [2]:


pclas_pct=pd.crosstab(titanic.Pclass.astype('category'),titanic.Survived.astype('category'),margins=True)



In [3]:
sex_pct=pd.crosstab(titanic.Sex.astype('category'),titanic.Survived.astype('category'),margins=True)

print(sex_pct)
""" % of survived """

sex_pct['percentage']=sex_pct[1]/(sex_pct[1]+sex_pct[0])

print(sex_pct)

titanic=pd.concat([titanic,pd.get_dummies(titanic['Sex'])],axis=1)
print(titanic)

Survived    0    1  All
Sex                    
female     64  195  259
male      360   93  453
All       424  288  712
Survived    0    1  All  percentage
Sex                                
female     64  195  259    0.752896
male      360   93  453    0.205298
All       424  288  712    0.404494
     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
885          886         0       3   
886          887         0       2   
887          888         1       1   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2     

In [4]:
'''Models created'''

X=titanic[['Pclass','Age','female']]
y=titanic[['Survived']]


In [5]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,random_state=45)

In [6]:
"""++++++++++++++++++++++++Decesion tree classifier+++++++++++++++++++++++++"""

'++++++++++++++++++++++++Decesion tree classifier+++++++++++++++++++++++++'

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,KFold
import numpy as np


model = DecisionTreeClassifier(criterion='entropy')
max_depth_val = np.arange(2,11)
min_samples_leaf_val = np.arange(1,31,9)
girdsearch_params ={'max_depth':max_depth_val,'min_samples_leaf':min_samples_leaf_val}

model_grid = GridSearchCV(estimator=model,param_grid=girdsearch_params,cv= KFold(n_splits=10))
model_grid.fit(X_train,y_train)

best_depth = model_grid.best_params_['max_depth']
best_min_samples_leaf = model_grid.best_params_['min_samples_leaf']

print("Best max_depth :"+str(best_depth)+" , \t best min_samples_leaf :"+str(+best_min_samples_leaf))




Best max_depth :7 , 	 best min_samples_leaf :10


In [37]:
"""CONFUSION MATRIX OF DATA"""


from sklearn.metrics import confusion_matrix
model=DecisionTreeClassifier(criterion='entropy',max_depth=best_depth,min_samples_leaf=best_min_samples_leaf)
TitanicTree=model.fit(X_train,y_train)
pred=TitanicTree.predict(X_test)
pred_proba=TitanicTree.predict_proba(X_test)
print("Confusion Matrix :")
print(confusion_matrix(y_test,pred))

print("Res Squre : %0.3f"%(+TitanicTree.score(X_test,y_test)))
print(type(TitanicTree.score(X_test,y_test)))

Confusion Matrix :
[[108   9]
 [ 39  58]]
Res Squre : 0.776
<class 'numpy.float64'>


In [43]:
"""Plotting the tree using Graphviz"""

from sklearn import tree

tree.export_graphviz(TitanicTree,out_file='TitanicTree.dot', max_depth=7,\
                     feature_names=X.columns,class_names=['Dead','Survived'])

from graphviz import Source

path = 'TitanicTree.dot'
s= Source.from_file(path)
s.view()

'TitanicTree.dot.pdf'

In [45]:
titanic_test = pd.read_csv('/home/jimbrootan/Desktop/ML_data/Titanic_Data/train.csv', header=0)

titanic_test = titanic_test.drop(['Ticket','Cabin'],axis=1)

titanic_test = titanic_test.dropna()

titanic_test = pd.concat([titanic_test,pd.get_dummies(titanic_test['Sex'])],axis=1)

titanic_test

X_test_data= titanic_test[['Pclass','Age','female']]

survive_count = TitanicTree.predict(X_test_data)
survive_count


array([0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,